<a href="https://colab.research.google.com/github/czhurdlespeed/TFDevCert/blob/main/TFCert_Practice/08_introduction_to_nlp_in_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to NLP Fundamentals in TensorFlow

## Check for GPU

In [2]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-a85d6e5e-5f98-96d9-fd0f-29c650d7d2c1)


## Import Helper Functions

In [4]:
!wget https://github.com/mrdbourke/tensorflow-deep-learning/raw/main/extras/helper_functions.py
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2024-01-27 12:55:37--  https://github.com/mrdbourke/tensorflow-deep-learning/raw/main/extras/helper_functions.py
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py [following]
--2024-01-27 12:55:37--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py.1’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2024-01-27 12:55:37 (83.0 MB/s) - ‘helper_fu

## Get a text dataset

- Using Kaggle's Introduction to NLP dataset

In [5]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
# Uzip data
unzip_data("nlp_getting_started.zip")

--2024-01-27 12:57:42--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.119.207, 108.177.111.207, 142.250.1.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.119.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.005s  

2024-01-27 12:57:42 (124 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Visualizing a text dataset

In [6]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [9]:
train_df["text"][100]

'.@NorwayMFA #Bahrain police had previously died in a road accident they were not killed by explosion https://t.co/gFJfgTodad'

In [10]:
# Shuffle training datafram
train_df_shuffled = train_df.sample(frac=1)
train_df_shuffled.head()

,id,keyword,location,text,target
5204,7431,obliterated,"Miami, FL",I can't wait to be beyond obliterated this wee...,0
3250,4671,engulfed,NaN,Men escape car engulfed in flames in Parley's ...,1
6969,9995,tsunami,"Gotham City,USA",I don't get waves of missing you anymore... Th...,0
2180,3124,debris,"Hamilton, Ontario Canada",Malaysia seem more certain than France.\n\nPla...,1
5505,7855,quarantined,California,Reddit's new content policy goes into effect m...,0


In [11]:
# What does the test dataframe look like?
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [12]:
# How many examples of each class are there?
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [13]:
# How many locations?
train_df.location.value_counts()

USA                    104
New York                71
United States           50
London                  45
Canada                  29
                      ... 
MontrÌ©al, QuÌ©bec       1
Montreal                 1
ÌÏT: 6.4682,3.18287      1
Live4Heed??              1
Lincoln                  1
Name: location, Length: 3341, dtype: int64

In [14]:
# How many total samples?
len(train_df), len(test_df)

(7613, 3263)

In [17]:
# Let's visualize some random training examples
import random
random_index = random.randint(0, len(train_df)-5)
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", r"(real disaster)" if target > 0 else r"(not real disaster)")
  print(f"Text:\n{text}\n")
  print("---\n")

Target: 0 (not real disaster)
Text:
@theburnageblue yes man i was having a bad week so far but Events + a deluge of favs have turned it right around

---

Target: 1 (real disaster)
Text:
.@CityofCalgary activates emergency plan amid severe thunderstorm warning http://t.co/pc7S8NxJ6Q #yyc #abstorm http://t.co/9xoHmMlMDY

---

Target: 1 (real disaster)
Text:
Blue Bell May Be Close to a Return From Its Listeria Disaster... Hot on #theneeds #Recipes http://t.co/F56v61AmPt

---

Target: 0 (not real disaster)
Text:
Owner of Chicago-Area Gay Bar Admits to Arson Scheme http://t.co/MYhOHvrHiL #LGBT | https://t.co/TM5HTHFDO0

---

Target: 0 (not real disaster)
Text:
RT to ruin @connormidd 's day.  http://t.co/krsy54XMMC

---



In [18]:
print("Hello\'\\ \n")

Hello'\ 



In [19]:
print(r"Hello\'\\ \n")

Hello\'\\ \n


### Split data into training and validation sets

In [21]:
from sklearn.model_selection import train_test_split
train_sentences, validation_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                                   train_df_shuffled["target"].to_numpy(),
                                                                                   test_size=0.1)


In [22]:
# Check the lengths
len(train_sentences), len(train_labels), len(validation_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [24]:
# Check the first 10 samples
train_sentences[:10], train_labels[:10]

(array(['Mayhem is beautiful',
        'Cuban leader extends sympathy to Vietnam over flooding at http://t.co/QcyXwr2rdv',
        '@CNN the End of Times are upon us. Famine War Death Plague. The presence is growing stronger.',
        'I want to go back to Vegas for my 21 but I feel like that would be such a disaster lol so much money would need to be brought',
        '@kakajambori ??\nU control the future of india..\nYor Subject: Exploration or seismic Maintenance( Electrical or Mechanical)',
        "@audacityjamesta Don't be like that babes &lt;3 you'll have a ball of fun with me at LAN! :)",
        "Russia destroys food while people go hungry.  We're not the only ones with an insane government.\nhttp://t.co/ZonNqGsxYw",
        'Bluedio Turbine Hurricane H Bluetooth 4.1 Wireless Stereo Headphones Headset BLK - Full re\x89Û_ http://t.co/WeUDLkc4o4 http://t.co/trl1dskF81',
        'For maximum damage! Activate [BIG BAND MODE] for old-timey obliteration!',
        '#ModiMinistry Ma